Tools  
[Write Midi Files](https://github.com/MarkCWirt/MIDIUtil)  
[Midi to CSV](https://pypi.org/project/py-midicsv/)  
[ABC Notation Player](https://github.com/mdoege/PySynth) 
[Midi to Wav](https://pypi.org/project/midi2audio/)  
[Mido <-> ABC in C](https://github.com/leesavide/abcmidi)  
[General Midi Instruments](https://en.wikipedia.org/wiki/General_MIDI#Bass)  


Resources  
[Andrej C-RNN Blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)  
[Midi File Format Info](https://www.csie.ntu.edu.tw/~r92092/ref/midi/)  
[Audio Stype Transfer Using Autoencoder](https://arxiv.org/abs/1812.07159)  



Architectures  
[C-RNN](https://github.com/karpathy/char-rnn)
[Autoencoder Example](https://www.learnopencv.com/understanding-autoencoders-using-tensorflow-python/)  


Tutorials  
[Siraj Uses Magenta](https://www.youtube.com/watch?v=pg9apmwf7og)  

MIDI Library  
[Cprato](https://www.cprato.com)  
[Midi File Library](https://bitmidi.com)  

Notes  
CaryKH added Mozart to Bach, which doubles sample size. He then tranposed all data 6 times, one for each semitone. Then modifed Adnrej's network to 3 layers and 700 neurons each. Transposing the music up a half step makes it look like a new music (in pitch) to the network [Link](https://www.youtube.com/watch?v=SacogDL_4JU)




In [1]:
# Tools
import os
import pandas as pd

# Play wav files
import IPython.display as ipd

# Play Midi Files

In [2]:
import pygame

def play_music1(music_file):
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print ("Music file %s loaded!" % music_file)
    except pygame.error:
        print ("File %s not found! (%s)" % (music_file, pygame.get_error()))
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        clock.tick(30)
        
def play_music(music_file):
    # Play Midi file
    try:
        play_music1(music_file)
    except KeyboardInterrupt:
        # if user hits Ctrl/C then exit
        # (works only in console mode)
        pygame.mixer.music.fadeout(1000)
        pygame.mixer.music.stop()
        raise SystemExit
        
def play_music_csv(music_csv):        
    # Parse the CSV output of the previous command back into a MIDI file
    midi_object = py_midicsv.csv_to_midi(music_csv)

    # Save the parsed MIDI file to disk
    with open("example_converted.mid", "wb") as output_file:
        midi_writer = py_midicsv.FileWriter(output_file)
        midi_writer.write(midi_object)
    play_music("example_converted.mid")

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## Run this code of midi player doesn't work

In [32]:
# Run this code of midi player doesn't work

midi_file = 'data/midi/'+df_edm['filenames'][0]
freq = 44100    # audio CD quality
bitsize = -16   # unsigned 16 bit
channels = 2    # 1 is mono, 2 is stereo
buffer = 1024    # number of samples
pygame.mixer.init(freq, bitsize, channels, buffer)

# optional volume 0 to 1.0
pygame.mixer.music.set_volume(0.8)
try:
    play_music(midi_file)
except KeyboardInterrupt:
    # if user hits Ctrl/C then exit
    # (works only in console mode)
    pygame.mixer.music.fadeout(1000)
    pygame.mixer.music.stop()
    raise SystemExit

Music file data/midi/Alan Walker - Alone  (midi by Carlo Prato) (www.cprato.com).mid loaded!


SystemExit: 

/Users/matthewlee/anaconda3/envs/nlp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Midi <-> CSV

## Compile data in pandas dataframe

In [80]:
import py_midicsv
import os

# Load file names
filenames = os.listdir('data/midi')
if '.DS_Store' in filenames:
    filenames.remove('.DS_Store')

filenames_clean = filenames.copy()
# Remove Unnecessary Strings
for ix,file in enumerate(filenames_clean):
    filenames_clean[ix] = filenames_clean[ix].replace('  (midi by Carlo Prato) (www.cprato.com)','')
    filenames_clean[ix] = filenames_clean[ix].replace(' (midi by Carlo Prato) (www.cprato.com)','')

# Generate Pandas Dataframe with filenames and CSV version of MIDI files
csvs = []
music_type = []
for filename in filenames:
    csvs.append(py_midicsv.midi_to_csv('data/midi/'+filename))
df_edm = pd.DataFrame({'filenames':filenames,'filenames_clean':filenames_clean,'csvs':csvs})
df_edm['filenames_clean2'] = df_edm['filenames_clean'].copy()
df_edm['filenames_clean2'] = df_edm['filenames_clean2'].apply(lambda x:x.upper())
df_edm.sort_values(by=['filenames_clean2'],inplace=True)
df_edm.drop(columns=['filenames_clean2'],inplace=True)
df_edm.reset_index(drop=True,inplace=True)

# # Parse the CSV output of the previous command back into a MIDI file
# midi_object = py_midicsv.csv_to_midi(csv_string)

# # Save the parsed MIDI file to disk
# with open("example_converted.mid", "wb") as output_file:
#     midi_writer = py_midicsv.FileWriter(output_file)
#     midi_writer.write(midi_object)

## Print indexes of instruments

In [6]:
for ix,song in enumerate(df_edm['csvs']):
    print(df_edm['filenames_clean'][ix]+' [index] : '+str(ix))
    for jx,row in enumerate(song):
        if 'Title' in row:
            print(jx,row)


Alan Walker - Alone.mid [index] : 0
8 3, 0, Title_t, "Lead"

336 4, 0, Title_t, "Bass"

640 5, 0, Title_t, "Drums"

878 6, 0, Title_t, "Alan Walker - Alone"

926 7, 0, Title_t, "Carlo Prato"

974 8, 0, Title_t, "cprato.com"

Alan Walker - Faded (Original Mix).mid [index] : 1
10 3, 0, Title_t, "Piano"

370 4, 0, Title_t, "Lead"

708 5, 0, Title_t, "Bass"

892 6, 0, Title_t, "Drums"

1138 7, 0, Title_t, "Alan Walker - Faded"

1194 8, 0, Title_t, "Carlo Prato"

1250 9, 0, Title_t, "cprato.com"

Alan Walker - Sing Me To Sleep.mid [index] : 2
8 3, 0, Title_t, "Lead"

312 4, 0, Title_t, "Bass"

440 5, 0, Title_t, "Drums"

1132 6, 0, Title_t, "Alan Walker - Sing Me To Sleep"

1180 7, 0, Title_t, "Carlo Prato"

1228 8, 0, Title_t, "cprato.com"

Calvin Harris - Blame.mid [index] : 3
8 3, 0, Title_t, "Lead"

344 4, 0, Title_t, "Bass"

648 5, 0, Title_t, "Drums"

886 6, 0, Title_t, "Calvin Harris - Blame"

934 7, 0, Title_t, "Carlo Prato"

982 8, 0, Title_t, "cprato.com"

Calvin Harris - My Way.m

## Print Header Info of All Songs

In [79]:
for i in range(len(instruments)):
    print(instruments[i][0])
    print('\n')

['0, 0, Header, 1, 8, 96\n', '1, 0, Start_track\n', '1, 0, Time_signature, 4, 2, 24, 8\n', '1, 0, End_track\n', '2, 0, Start_track\n', '2, 0, Tempo, 618557\n', '2, 0, End_track\n']


['0, 0, Header, 1, 9, 96\n', '1, 0, Start_track\n', '1, 0, Time_signature, 4, 2, 24, 8\n', '1, 0, End_track\n', '2, 0, Start_track\n', '2, 0, Tempo, 666667\n', '2, 0, Tempo, 666667\n', '2, 0, Tempo, 666667\n', '2, 0, End_track\n']


['0, 0, Header, 1, 8, 96\n', '1, 0, Start_track\n', '1, 0, Time_signature, 4, 2, 24, 8\n', '1, 0, End_track\n', '2, 0, Start_track\n', '2, 0, Tempo, 681818\n', '2, 0, End_track\n']


['0, 0, Header, 1, 8, 96\n', '1, 0, Start_track\n', '1, 0, Time_signature, 4, 2, 24, 8\n', '1, 0, End_track\n', '2, 0, Start_track\n', '2, 0, Tempo, 468750\n', '2, 0, End_track\n']


['0, 0, Header, 1, 9, 96\n', '1, 0, Start_track\n', '1, 0, Time_signature, 4, 2, 24, 8\n', '1, 0, End_track\n', '2, 0, Start_track\n', '2, 0, Tempo, 500000\n', '2, 0, End_track\n']


['0, 0, Header, 1, 9, 96\n', '1, 0,

## Parse instruments for each song

In [81]:
# For each song, break it up by instruments
instruments = []
for ix,song in enumerate(df_edm['csvs']):
    holder = []
    indexes = []
    #holder.append(df_edm['filenames_clean'][ix]+' [index] : '+str(ix))
    for jx,row in enumerate(song):
        if 'Title_t' in row:
            indexes.append(jx-1)
    # Header and info before first instrument
    holder.append(song[0:indexes[0]])
    # Add each instrument for each song
    for i in range(len(indexes)-1):
        holder.append(song[indexes[i]:indexes[i+1]])
    instruments.append(holder) 

# Index of instruments that actually has notes. This is needed to get rid of instruments that doesn't have any notes
for ix,song in enumerate(instruments):
    keep_instrument_index = []
    for jx,instrument in enumerate(song[1:]):
        note_on_off = -1
        for kx,row in enumerate(instrument):
            note_on_off = row.find('Note_on')
            if note_on_off!=-1:
                keep_instrument_index.append(jx+1)
                break
    instruments[ix]=instruments[ix][:keep_instrument_index[-1]+1]

# Extract only note_on and note_off infor for melodies, bass, and drum
melody_name = ['Bassmelody','Chords','Lead','MIDI Out','Pad','Piano','Saw Chords','Second Lead','Strings',
'Voice','Voice Synth']
bass_name = ['Basses','basses','Bass','bass']
drum_name = ['Drums','drums','Drum','drum']
melody, bass, drum = [], [], []
for ix,song in enumerate(instruments):
    melody_holder, bass_holder, drum_holder = [], [], []
    for jx,instrument in enumerate(song[1:]):
        if instrument[1].split('"')[1] in bass_name:
            first_actual_note_indexes = []
            for ix,seq in enumerate(instrument):
                if ('Note_on_c' in seq) or ('Note_off_c' in seq):
                    first_actual_note_indexes.append(ix)
            bass_holder.append(instrument[first_actual_note_indexes[0]:first_actual_note_indexes[-1]+1])  
        elif instrument[1].split('"')[1] in drum_name:
            first_actual_note_indexes = []
            for ix,seq in enumerate(instrument):
                if ('Note_on_c' in seq) or ('Note_off_c' in seq):
                    first_actual_note_indexes.append(ix)
            drum_holder.append(instrument[first_actual_note_indexes[0]:first_actual_note_indexes[-1]+1])             
        else:
            first_actual_note_indexes = []
            for ix,seq in enumerate(instrument):
                if ('Note_on_c' in seq) or ('Note_off_c' in seq):
                    first_actual_note_indexes.append(ix)
            melody_holder.append(instrument[first_actual_note_indexes[0]:first_actual_note_indexes[-1]+1])
    melody.append(melody_holder)
    bass.append(bass_holder)
    drum.append(drum_holder)
    

# # Get rid of any 'Control_c','Pitch_bend_c','Program_c'
# start_end_track = []
# for ix,song in enumerate(instruments):    
#     for jx,instrument in enumerate(song):
#         remove_control_pitch_program = []
#         for kx,row in enumerate(instrument):
#             if row.split(', ')[2] not in ['Control_c','Pitch_bend_c','Program_c']:
#                 remove_control_pitch_program.append(row)
#         instruments[ix][jx] = remove_control_pitch_program
        
        

In [ ]:


# Split each note for easier manipulation
for ix,note in enumerate(first_instrument):
    first_instrument[ix] = first_instrument[ix].split(', ')

## Play 

In [45]:
play_music_csv(df_edm['csvs'][21])

Music file example_converted.mid loaded!


In [44]:
song_index = 21
instrument_concat = []
for instrument in instruments[song_index]:
    instrument_concat+=instrument
play_music_csv(instrument_concat)

Music file example_converted.mid loaded!


SystemExit: 

In [ ]:
first_actual_note_indexes = []
for ix,seq in enumerate(instruments[19][2]):
    if ('Note_on_c' in seq) or ('Note_off_c' in seq):
        first_actual_note_indexes.append(ix)
first_instrument = instruments[19][2][first_actual_note_indexes[0]:first_actual_note_indexes[-1]+1]

# Split each note for easier manipulation
for ix,note in enumerate(first_instrument):
    first_instrument[ix] = first_instrument[ix].split(', ')

start_track = instruments[19][2][:2]
program = instruments[19][2][first_actual_note_indexes[0]-1]
end_track = instruments[19][2][-1]
    

In [11]:
# Combine different melody instruments into one melody instrument

# Combine two instruments
df_combined_instrument = pd.DataFrame(first_instrument+second_instrument+third_instrument)
# Timestamp column as integer
df_combined_instrument[1] = df_combined_instrument[1].astype('int32')
# Set the two instruments as same instrument number
df_combined_instrument[0]=3
# Set the two instruments as same instrument type
df_combined_instrument[3]=12
# Reorder df_combined_instrument by timestamp & drop duplicates
df_combined_instrument_ordered = df_combined_instrument.sort_values(by=[1]).reset_index(drop=True).copy()
df_combined_instrument_ordered.drop_duplicates(inplace=True)
# Reconstruct ordered data into csv format to be turned into midi file
combined_instrument_ordered = []
for i in range(len(df_combined_instrument_ordered)):
    combined_instrument_ordered.append(', '.join([str(x) for x in df_combined_instrument_ordered.iloc[i].tolist()]))

In [19]:
# Need the 'End_track' row at the end of the file so that Midi player works
play_music_csv(instruments[19][1]+instruments[19][2][0:2]+combined_instrument_ordered+['3, 6529, End_track\n']+asdf)

Music file example_converted.mid loaded!


SystemExit: 

# Download Youtube

In [156]:
# Use below command in the terminal to download youtube file as wav file.
#!python YouTube_to_WAV.py https://www.youtube.com/watch?v=xgs-laNZ0SE&t=218s

# Midi -> Wav (Doesn't work atm)

In [17]:
# #!pip install midi2audio --y
from midi2audio import FluidSynth

# using the default sound font in 44100 Hz sample rate
fs = FluidSynth()
fs.midi_to_audio('Alan Walker - Alone  (midi by Carlo Prato) (www.cprato.com) copy.mid','output.wav')
# for ix,filename in enumerate(filenames):
#     fs.midi_to_audio('data/midi/'+filename, 'data/wav/'+df_edm['filenames_clean'][ix][:-4]+'.wav')


# Play ABC Notation

In [ ]:
# ABC Notation Player
import pysynth as ps

# Make test ABC file
test = (
  ('g#', 4),  ('g*', 2), ('g5', 4),
  ('g5*', 4), ('r', 4), ('e5', 16),
  ('f5', 16),  ('e5', 16),  ('d5', 16),
  ('e5*', 4) 
)
ps.make_wav(test, fn = "test.wav")

#Play wav file
ipd.Audio('test.wav') # load a local WAV file